In [ ]:
import nltk
import numpy as np

In [4]:
#!pip install nltk

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

## Functions

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    """
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    sentence_words = [stem(word) for word in tokenized_sentence]      # stem each word
    bag = np.zeros(len(words), dtype=np.float32)                     # initialize bag with 0 for each word
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1
            
    return bag

## Data Processing

In [ ]:
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [13]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [14]:
all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [17]:
print(all_words)

["'s", '10', 'a', 'about', 'accept', 'address', 'afternoon', 'an', 'are', 'at', 'book', 'bye', 'call', 'can', 'card', 'cash', 'close', 'come', 'contact', 'credit', 'day', 'deliv', 'deliveri', 'describ', 'detail', 'dinner', 'direct', 'dish', 'do', 'doe', 'even', 'food', 'for', 'friday', 'get', 'give', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'hour', 'how', 'i', 'in', 'is', 'kind', 'later', 'like', 'locat', 'long', 'lot', 'make', 'mastercard', 'me', 'menu', 'morn', 'my', 'number', 'of', 'offer', 'on', 'onli', 'open', 'option', 'order', 'pay', 'paypal', 'phone', 'pick', 'place', 'pm', 'reach', 'reserv', 'restaur', 'schedul', 'see', 'serv', 'servic', 'show', 'some', 'tabl', 'take', 'takeout', 'tell', 'thank', 'that', 'the', 'time', 'to', 'two', 'up', 'vegan', 'want', 'websit', 'weekend', 'what', 'whatâ€™', 'when', 'where', 'with', 'you', 'your']


In [18]:
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]       # stem and lower

all_words = sorted(set(all_words))    # remove duplicates and sort

## Dataset

In [40]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(all_words)
hidden_size = 10
output_size = len(tags)

In [21]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

In [22]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [24]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [25]:
dataset = ChatDataset()

In [28]:
dataset.__getitem__(4)

(array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.], dtype=float32),
 0)

In [30]:
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

## Model

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
class model_cb(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(model_cb, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.relu(x)
        x = self.l3(x)
        # no activation and no softmax at the end
        return x

In [55]:
model = model_cb(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()     # Loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)     #optimizer

## training loop

In [56]:
#traininf loop
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.5582
Epoch [200/1000], Loss: 0.0415
Epoch [300/1000], Loss: 0.0292
Epoch [400/1000], Loss: 0.0021
Epoch [500/1000], Loss: 0.0012
Epoch [600/1000], Loss: 0.0011
Epoch [700/1000], Loss: 0.0002
Epoch [800/1000], Loss: 0.0001
Epoch [900/1000], Loss: 0.0002
Epoch [1000/1000], Loss: 0.0001
final loss: 0.0001


## Model Save

In [57]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

torch.save(data, "model_data.pth")

## Chat Bot

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

In [ ]:
data = torch.load("model_data.pth")

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

In [ ]:
model = model_cb(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

In [ ]:
bot_name = "Lexa"

print("Let's chat! (type 'quit' to exit)")

while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand you.")

# -------------------------------END-------------------------------

## workspace

In [66]:
output.shape

torch.Size([1, 11])

In [67]:
output

tensor([[-4.1753,  1.6946, -9.4218, -7.6858, -4.4118, -5.9664, -7.6212, -6.4075,
         -7.3662, -6.9610, -7.4906]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [68]:
predicted

tensor([1], device='cuda:0')

In [69]:
predicted.item()

1

In [73]:
torch.max(output, dim=1)[0]

tensor([1.6946], device='cuda:0', grad_fn=<MaxBackward0>)

In [76]:
torch.softmax(output, dim=1)

tensor([[2.8052e-03, 9.9363e-01, 1.4772e-05, 8.3823e-05, 2.2143e-03, 4.6784e-04,
         8.9421e-05, 3.0098e-04, 1.1539e-04, 1.7304e-04, 1.0190e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [77]:
probs[0]

tensor([2.8052e-03, 9.9363e-01, 1.4772e-05, 8.3823e-05, 2.2143e-03, 4.6784e-04,
        8.9421e-05, 3.0098e-04, 1.1539e-04, 1.7304e-04, 1.0190e-04],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [78]:
probs[0][predicted.item()]

tensor(0.9936, device='cuda:0', grad_fn=<SelectBackward0>)

In [79]:
X

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],
       device='cuda:0')